In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('./gdrive/MyDrive/AEGAN/AE_GAN_CS')
!nvidia-smi
!ls

In [ ]:
!pip install adversarial-robustness-toolbox

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms
import numpy as np
from art.attacks.evasion import FastGradientMethod
from art.attacks.evasion import CarliniL2Method
from art.attacks.evasion import DeepFool
from art.attacks.evasion import BasicIterativeMethod
from art.attacks.evasion import SaliencyMapMethod
from art.estimators.classification import PyTorchClassifier
from art.defences.preprocessor import FeatureSqueezing
from art.defences.preprocessor import TotalVarMin
from art.defences.preprocessor import SpatialSmoothing
from art.defences.preprocessor import JpegCompression

In [ ]:
def double_conv(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, 3, padding=1),
        nn.ReLU(inplace=True)
    )   

# mnist
class Generator2(nn.Module):
    def __init__(self, in_ch):
        super(Generator2, self).__init__()      
        self.dconv_down1 = double_conv(in_ch, 64)
        self.dconv_down2 = double_conv(64, 128)
        self.dconv_down3 = double_conv(128, 256)
        self.maxpool = nn.MaxPool2d(2)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)        
        self.dconv_up3 = double_conv(128 + 256, 128)
        self.dconv_up2 = double_conv(64 + 128, 64)    
        self.conv_last = nn.Conv2d(64, in_ch, 1)
              
    def forward(self, x):
        conv1 = self.dconv_down1(x)
        x = self.maxpool(conv1)
        conv2 = self.dconv_down2(x)
        x = self.maxpool(conv2)
        x = self.dconv_down3(x)
        x = self.upsample(x)        
        x = torch.cat([x, conv2], dim=1)
        x = self.dconv_up3(x)
        x = self.upsample(x)        
        x = torch.cat([x, conv1], dim=1)              
        x = self.dconv_up2(x)   
        out = self.conv_last(x)   
        return out

1-8,12,13

In [ ]:
G1 = Generator2(1).cuda()
G1.load_state_dict(torch.load("./checkpoint/mnist/1.tar")["generator"])
G1.eval()

G2 = Generator2(1).cuda()
G2.load_state_dict(torch.load("./checkpoint/mnist/2.tar")["generator"])
G2.eval()

G3 = Generator2(1).cuda()
G3.load_state_dict(torch.load("./checkpoint/mnist/3.tar")["generator"])
G3.eval()

G4 = Generator2(1).cuda()
G4.load_state_dict(torch.load("./checkpoint/mnist/4.tar")["generator"])
G4.eval()

G5 = Generator2(1).cuda()
G5.load_state_dict(torch.load("./checkpoint/mnist/5.tar")["generator"])
G5.eval()

G6 = Generator2(1).cuda()
G6.load_state_dict(torch.load("./checkpoint/mnist/6.tar")["generator"])
G6.eval()

G7 = Generator2(1).cuda()
G7.load_state_dict(torch.load("./checkpoint/mnist/7.tar")["generator"])
G7.eval()

G8 = Generator2(1).cuda()
G8.load_state_dict(torch.load("./checkpoint/mnist/8.tar")["generator"])
G8.eval()

G9 = Generator2(1).cuda()
G9.load_state_dict(torch.load("./checkpoint/mnist/12.tar")["generator"])
G9.eval()

G10 = Generator2(1).cuda()
G10.load_state_dict(torch.load("./checkpoint/mnist/13.tar")["generator"])
G10.eval()

In [ ]:
class Generator(nn.Module):

    def __init__(self, in_ch):
        super(Generator, self).__init__()
        self.conv1 = nn.Conv2d(in_ch, 64, 4, stride=2, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, 4, stride=2, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.deconv3 = nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.deconv4 = nn.ConvTranspose2d(64, in_ch, 4, stride=2, padding=1)

    def forward(self, x):
        h = F.leaky_relu(self.bn1(self.conv1(x)))
        h = F.leaky_relu(self.bn2(self.conv2(h)))
        h = F.leaky_relu(self.bn3(self.deconv3(h)))
        h = torch.tanh(self.deconv4(h))
        return h

In [ ]:
G11 = Generator(1).cuda()
G11.load_state_dict(torch.load("./APEGAN/mnist/3.tar")["generator"])
G11.eval()

In [ ]:
test_loader = torch.utils.data.DataLoader(
            torchvision.datasets.MNIST(root = './', train=False, download=False,
                           transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor()])),
            batch_size=128, shuffle=False)

In [ ]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(1, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])


def ResNet50():
    return ResNet(Bottleneck, [3, 4, 6, 3])


In [ ]:
model = ResNet18().cuda()
model.load_state_dict(torch.load("./resnet/mnist_resNet18.tar")["state_dict"])
model.eval()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
classifier = PyTorchClassifier(
    model=model,
    clip_values=(0, 1),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(1, 28, 28),
    nb_classes=10,
)

In [ ]:
accuracy = 0
for x_test, y_test in test_loader:
  predictions = classifier.predict(x_test)
  accuracy += np.sum(np.argmax(predictions, axis=1) == y_test.numpy())
print('Accuracy on benign test examples:{}%'.format(accuracy/10000*100))

In [ ]:
# attack_fgsm = FastGradientMethod(estimator=classifier,eps=0.1)
# attack_fgsm = BasicIterativeMethod(estimator=classifier, eps=0.7)
# attack_fgsm = SaliencyMapMethod(classifier=classifier, theta=0.3, gamma=0.8)
# attack_fgsm = DeepFool(classifier=classifier)
attack_fgsm = CarliniL2Method(classifier=classifier, confidence=0.0001,binary_search_steps=25, max_iter=25)
adv_fgsm_accuracy = 0
re_accuracy = 0
num = 0
for x_test, y_test in test_loader:
  adv_fgsm = attack_fgsm.generate(x=x_test)
  torch.save({"CW":adv_fgsm,"y":y_test}, "CW.tar")
  adv_fgsm_predictions = classifier.predict(adv_fgsm)
  adv_fgsm_accuracy += np.sum(np.argmax(adv_fgsm_predictions, axis=1) == y_test.numpy())

  re_x1 = G1(torch.from_numpy(adv_fgsm).cuda())
  re_y1 = classifier.predict(re_x1.cpu().detach().numpy())

  re_x2 = G2(torch.from_numpy(adv_fgsm).cuda())
  re_y2 = classifier.predict(re_x2.cpu().detach().numpy())

  re_x3 = G3(torch.from_numpy(adv_fgsm).cuda())
  re_y3 = classifier.predict(re_x3.cpu().detach().numpy())

  re_x4 = G4(torch.from_numpy(adv_fgsm).cuda())
  re_y4 = classifier.predict(re_x4.cpu().detach().numpy())

  re_x5 = G5(torch.from_numpy(adv_fgsm).cuda())
  re_y5 = classifier.predict(re_x5.cpu().detach().numpy())

  re_x6 = G6(torch.from_numpy(adv_fgsm).cuda())
  re_y6 = classifier.predict(re_x6.cpu().detach().numpy())

  re_x7 = G7(torch.from_numpy(adv_fgsm).cuda())
  re_y7 = classifier.predict(re_x7.cpu().detach().numpy())

  re_x8 = G8(torch.from_numpy(adv_fgsm).cuda())
  re_y8 = classifier.predict(re_x8.cpu().detach().numpy())

  re_x9 = G9(torch.from_numpy(adv_fgsm).cuda())
  re_y9 = classifier.predict(re_x9.cpu().detach().numpy())

  re_x10 = G10(torch.from_numpy(adv_fgsm).cuda())
  re_y10 = classifier.predict(re_x10.cpu().detach().numpy())

  re_y = re_y10

  # APE-GAN
  # re_x = G11(torch.from_numpy(adv_fgsm).cuda())
  # re_y = classifier.predict(re_x.cpu().detach().numpy())
  # bit_depth
  # defense0 = FeatureSqueezing(clip_values=(0.0,0.8),bit_depth=1)
  # re_x = defense0(adv_fgsm)[0]
  # re_y = classifier.predict(re_x)
  # TotalVarMin
  # defense2 = TotalVarMin()
  # re_x = defense2(adv_fgsm)[0]
  # re_y = classifier.predict(re_x)
  # SpatialSmoothing
  # defense3 = SpatialSmoothing(window_size=2)
  # re_x = defense3(adv_fgsm)[0]
  # re_y = classifier.predict(re_x)
  # JpegCompression
  # defense4 = JpegCompression(clip_values=(0.0,1.0))
  # re_x = defense4(adv_fgsm)[0]
  # re_y = classifier.predict(re_x)


  re_accuracy += np.sum(np.argmax(re_y, axis=1) == y_test.numpy())
  num = num + len(x_test)
  print(adv_fgsm_accuracy/num*100)
  print(re_accuracy/num*100)
  if num>100:
    break


0.78125
63.28125


In [ ]:
cw = torch.load('./CW.tar')
adv_fgsm = cw['CW']
y_test = cw['y']
adv_fgsm_predictions = classifier.predict(adv_fgsm)
adv_fgsm_accuracy = np.sum(np.argmax(adv_fgsm_predictions, axis=1) == y_test.numpy())

re_x1 = G1(torch.from_numpy(adv_fgsm).cuda())
re_y1 = classifier.predict(re_x1.cpu().detach().numpy())

re_x2 = G2(torch.from_numpy(adv_fgsm).cuda())
re_y2 = classifier.predict(re_x2.cpu().detach().numpy())

re_x3 = G3(torch.from_numpy(adv_fgsm).cuda())
re_y3 = classifier.predict(re_x3.cpu().detach().numpy())

re_x4 = G4(torch.from_numpy(adv_fgsm).cuda())
re_y4 = classifier.predict(re_x4.cpu().detach().numpy())

re_x5 = G5(torch.from_numpy(adv_fgsm).cuda())
re_y5 = classifier.predict(re_x5.cpu().detach().numpy())

re_x6 = G6(torch.from_numpy(adv_fgsm).cuda())
re_y6 = classifier.predict(re_x6.cpu().detach().numpy())

re_x7 = G7(torch.from_numpy(adv_fgsm).cuda())
re_y7 = classifier.predict(re_x7.cpu().detach().numpy())

re_x8 = G8(torch.from_numpy(adv_fgsm).cuda())
re_y8 = classifier.predict(re_x8.cpu().detach().numpy())

re_x9 = G9(torch.from_numpy(adv_fgsm).cuda())
re_y9 = classifier.predict(re_x9.cpu().detach().numpy())

re_x10 = G10(torch.from_numpy(adv_fgsm).cuda())
re_y10 = classifier.predict(re_x10.cpu().detach().numpy())


re_y = re_y10

# APE-GAN
# re_x = G(torch.from_numpy(adv_fgsm).cuda())
# re_y = classifier.predict(re_x.cpu().detach().numpy())
# bit_depth
# defense0 = FeatureSqueezing(clip_values=(0.0,0.8),bit_depth=1)
# re_x = defense0(adv_fgsm)[0]
# re_y = classifier.predict(re_x)
# TotalVarMin
# defense2 = TotalVarMin()
# re_x = defense2(adv_fgsm)[0]
# re_y = classifier.predict(re_x)
# SpatialSmoothing
# defense3 = SpatialSmoothing(window_size=2)
# re_x = defense3(adv_fgsm)[0]
# re_y = classifier.predict(re_x)

re_accuracy = np.sum(np.argmax(re_y, axis=1) == y_test.numpy())
print(adv_fgsm_accuracy/len(y_test)*100)
print(re_accuracy/len(y_test)*100)

0.78125
63.28125
